In [4]:
import pandas as pd
from tqdm import tqdm
%run slicer.ipynb
%run ts_to_intervals.ipynb
%run dataset_to_slices.ipynb

                                  hs                        he
main_value                                                    
p01        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p02        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p03        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p04        2019-11-01 00:00:00+00:00 2020-03-31 17:12:00+00:00
p05        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p06        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p07        2019-11-05 00:00:00+00:00 2020-03-31 18:55:00+00:00
p08        2019-11-10 00:00:00+00:00 2020-03-31 23:59:00+00:00
p09        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p10        2019-11-05 00:00:00+00:00 2020-03-31 23:59:00+00:00
p11        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p12        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p13        2019-11-01 00:00:00+00:00 2020-03-31 23:59:00+00:00
p14        2019-11-13 00:00:00+00:00 2020-03-31 23:30:3

### samples_from_ds_ts
creates two dataframes from time series data:<br>
odfs: from start till waiting window.<br>
pdfs: from waiting window till prediction window dataframes.

In [31]:
def samples_from_df_ts(
        dataframe, hs, he, 
        main_feature,  # "participant"
        time_feature,  # "vt" -> "seconds" before calling create_buckets
        group_feature,  # "calories"
        main_value,   # "p01"
        slicer_params,  # { "stride":3,  "ow":3, "ww":6, "pw":14, "granularity":'hours'}
        bucket_params):
    
    df = dataframe[dataframe[main_feature] == main_value].reset_index().copy()
    slices = slicer(hs,he, **slicer_params)
    st_ow_dfs = [df[(slices.iloc[i].st <= df[time_feature]) & (df[time_feature] <= slices.iloc[i].ow)].copy() for i in range(len(slices))]
    ww_pw_dfs = [df[(slices.iloc[i].ww <= df[time_feature]) & (df[time_feature] <= slices.iloc[i].pw)].copy() for i in range(len(slices))]

    for i, ww_pw_df in enumerate(tqdm(ww_pw_dfs)):
        ww_pw_df["i"] = i
    r = []    
    for i, st_ow_df in enumerate(tqdm(st_ow_dfs)):
        transaction_start = slices.iloc[i].st
        st_ow_df["seconds"]  = (st_ow_df[time_feature] - transaction_start).dt.total_seconds() #.apply(lambda x: x.total_seconds())
        bucket_values = bucket_params["bucket_values"]
        bi = create_intervals(create_buckets(st_ow_df,bucket_values, group_feature, "seconds", group_feature))
        bi["i"] = i
        bi["vt"] = bi["interval"].apply(lambda x: x[0])
        bi["value"] = bi["interval"].apply(lambda x: x[1] - x[0])
        bi["label"] = bi["bucket"]
        bi = bi[["i", "vt", "label", "value"]].copy()
        r.append(bi)
    return r, ww_pw_dfs

histories = histories_df
p = "p01"
feature_csv ="calories.csv"
df_cal = pd.read_csv(f'datasets/fitbit/processed_data/{feature_csv}', parse_dates=["vt"])
df_cal.vt = df_cal.vt.apply(lambda x: x.tz_localize(None).tz_localize('UTC'))



odfs, pdfs = samples_from_df_ts(df_cal, *tuple(histories.loc[p]), "participant","vt", "calories", p,
            {"stride":24,  "ow":12, "ww":12, "pw":14, "granularity":'hours'}, {"bucket_values": [1.40, 4.50, 10]})


  8%|▊         | 12/151 [00:02<00:30,  4.60it/s]/Users/omid/mambaforge/lib/python3.10/site-packages/sklearn/cluster/_mean_shift.py:271: UserWarning:

Binning data failed with provided bin_size=85.714286, using data points as seeds.

 30%|███       | 46/151 [00:08<00:19,  5.41it/s]/Users/omid/mambaforge/lib/python3.10/site-packages/sklearn/cluster/_mean_shift.py:271: UserWarning:

Binning data failed with provided bin_size=60.000000, using data points as seeds.

 54%|█████▎    | 81/151 [00:13<00:10,  6.59it/s]/Users/omid/mambaforge/lib/python3.10/site-packages/sklearn/cluster/_mean_shift.py:271: UserWarning:

Binning data failed with provided bin_size=60.000000, using data points as seeds.

 74%|███████▍  | 112/151 [00:19<00:07,  5.15it/s]/Users/omid/mambaforge/lib/python3.10/site-packages/sklearn/cluster/_mean_shift.py:271: UserWarning:

Binning data failed with provided bin_size=60.000000, using data points as seeds.

 91%|█████████ | 137/151 [00:22<00:01,  8.19it/s]/Users/omid/mambaf

In [32]:
odfs[1].sort_values("vt")

,i,vt,label,value
0,1,0.0,calories_1,13560.0
3,1,240.0,calories_2,2220.0
4,1,10500.0,calories_2,14280.0
1,1,13620.0,calories_1,15060.0
5,1,26040.0,calories_2,16920.0
6,1,28260.0,calories_3,1920.0
10,1,29280.0,calories_4,0.0
11,1,29340.0,calories_4,0.0
7,1,31080.0,calories_3,2280.0
2,1,33840.0,calories_1,8880.0


### samples_from_df_ev
creates two dataframes from event databases:<br>
odfs: from start till waiting window.<br>
pdfs: from waiting window till prediction window dataframes.<br>
it does the same thing as "samples_from_df_ev" but doesn't do any bucketing.

In [7]:
def samples_from_df_ev(dataframe, #"exercise.csv as example"
        hs, # history start
        he, # history end
        main_feature,  # "participant"
        time_feature,  # "vt" -> then adding "seconds" columns before calling create_buckets
        group_feature,  # "Activity"
        value_feature, # "value"--new-- and lacks bucket_params
        main_value,   # "p01"
        slicer_params): # { "stride":3,  "ow":3, "ww":6, "pw":14, "granularity":'hours'})  
    
    df = dataframe[dataframe[main_feature] == main_value].reset_index().copy()
    slices = slicer(hs, he, **slicer_params)
    
    st_ow_dfs = [df[(slices.iloc[i].st <= df[time_feature]) & (df[time_feature] <= slices.iloc[i].ow)].copy() for i in range(len(slices))]
    ww_pw_dfs = [df[(slices.iloc[i].ww <= df[time_feature]) & (df[time_feature] <= slices.iloc[i].pw)].copy() for i in range(len(slices))]
    
    for i, ww_pw_df in enumerate(tqdm(ww_pw_dfs)):
        ww_pw_df["i"] = i
    
    r = []    
    for i, st_ow_df in enumerate(tqdm(st_ow_dfs)):
        transaction_start = slices.iloc[i].st
        if not st_ow_df.empty:
            #st_ow_df["vt"] = (st_ow_df[time_feature] - min(st_ow_df[time_feature])).apply(lambda x: x.total_seconds())
            st_ow_df["vt"] = (st_ow_df[time_feature] - transaction_start).dt.total_seconds()


        bi = st_ow_df[[time_feature, group_feature, value_feature]].copy()
        bi["i"] = i
        bi["vt"] = bi["vt"] #or can be time_feature !
        bi["label"] = bi[group_feature]
        bi["value"] = bi[value_feature]
        bi = bi[["i", "vt", "label", "value"]].copy()
        r.append(bi)
    
    return r, ww_pw_dfs

# Example usage:
histories = histories_df
p = "p01"
feature_csv = "exercise.csv"
df_ex = pd.read_csv(f'datasets/fitbit/processed_data/{feature_csv}', parse_dates=["vt"])
df_ex.vt = df_ex.vt.apply(lambda x: x.tz_localize(None).tz_localize('UTC'))

odfs_ev, pdfs_ev = samples_from_df_ev(df_ex, *tuple(histories.loc[p]), "participant", "vt", "Activity", "value", p,
                                       {"stride": 24, "ow": 12, "ww": 12, "pw": 14, "granularity": 'hours'})


100%|██████████| 151/151 [00:00<00:00, 1440.33it/s]


In [8]:
odfs_ev[112]

,i,vt,label,value
151,112,24855.0,Run,1240000
152,112,35317.0,Run,2102000


#### Getting Info About all the files in the folder

In [9]:
import os

def info_csv_files(folder_path):

    # Gets a list of all CSV files
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for file in csv_files:
        file_path = os.path.join(folder_path, file)

        df = pd.read_csv(file_path)

        print(f"\nFile: {file}\nColumn Names: {', '.join(df.columns)}")

        print(df.describe())

folder_path = 'datasets/fitbit/processed_data/'
info_csv_files(folder_path)



File: sleep_score.csv
Column Names: participant, vt, label, value
             value
count  1836.000000
mean     76.460784
std       7.795554
min      35.000000
25%      72.000000
50%      77.000000
75%      82.000000
max      94.000000

File: participant.csv
Column Names: ParticipantId, Age, Height, Gender, AOrBPerson, MaxHeartRate, FirstFiveKmRunDate, Minutes, Seconds, StrideWalk, StrideRun
             Age      Height  StrideWalk   StrideRun
count  16.000000   16.000000   14.000000   14.000000
mean   34.875000  179.625000   74.230714   97.528571
std    11.672618    6.781101    3.102040   27.019606
min    23.000000  163.000000   67.300000   11.300000
25%    26.000000  177.750000   73.052500   95.125000
50%    29.000000  180.000000   74.700000  102.600000
75%    42.750000  182.250000   75.400000  108.825000
max    60.000000  195.000000   80.900000  128.800000

File: exercise.csv
Column Names: participant, vt, Activity, value
              value
count  2.440000e+03
mean   2.516525e+06

### SAMPLES_FROM_DATASET function

##### samples_from_dataset (<br>
       main_value, # "p01" <br>
       timeseries_df_list, # [{“path":"here goes path of all the timeseries dataframes" , "main_feature":"e.g. participant",
        #"time_feature":"e.g. vt", "group_feature":"e.g. Activity", "bucket_values":"[1.40, 4.50, 10]"}],
       event_df_list, # [{“path":"here goes path of all the event dataframes", "main_feature":"e.g. participant", #"time_feature":"e.g. vt",
       #"group_feature":"e.g. Activity","value_feature":"e.g. value"}]
       slicer_params #[{"vts: should be calculated with dataset_history function(hs)" ,"vte":"should be calculated with dataset_history function(he)",
       #"stride":24,  "ow":12, "ww":12, "pw":14, "granularity":"hours"}]
):<br>
    # steps of this function:<br>
    # a- computes the histories (hs,he) on all the .csv files.<br>
    # b- loads the file in datframe<br>
    # c- fix “vt“s timezone  <br>
    # d- generates the dataframes with either samples_from_df_ts or samples_from_df_ev functions.<br>
    # e- concatenates all the returned "odfs" and "odfs_ev" ordered, in a dataframe. Concatenates the list of "pdfs" and "pdfs_ev" in a dataframe.<br>
    pass



In [10]:
def samples_from_dataset(main_value, main_feature, time_feature, timeseries_data, event_data, slicer_params):
    
    all_odfs, all_pdfs = [], {}

    history = dataset_history(timeseries_data["paths"]+event_data["paths"], main_feature, time_feature, main_value)

    for path, group_feature, bucket_values in zip(timeseries_data["paths"], timeseries_data["group_features"], timeseries_data["bucket_values"]):
            # Step a: Compute histories (hs, he) on all the .csv files

            # Step b & c: Load the file in a DataFrame and fix time_feature's timezone
        ts_df = pd.read_csv(path, parse_dates=[time_feature])
        ts_df[time_feature] = ts_df[time_feature].apply(lambda x: x.tz_localize(None).tz_localize('UTC'))

            # Step d.a timeseries: Generate dataframes with samples_from_df_ts function
        t_odfs, t_pdfs = samples_from_df_ts(
                ts_df, history[0], history[1], main_feature,
                time_feature, group_feature,
                main_value, slicer_params, {"bucket_values": bucket_values}
            )
            #Iterates over its argument and adding each element to the list and extending the list,O(k)(used instead of append,O(1))
        all_odfs.extend(t_odfs)
        all_pdfs[group_feature] = t_pdfs

    for path, group_feature, value_feature in zip(event_data["paths"], event_data["group_features"], event_data["value_feature"]):
            # Step a: Compute histories (hs, he) on all the .csv files

            # Step b & c: Load the file in a DataFrame and fix time_feature's timezone
        ev_df = pd.read_csv(path, parse_dates=[time_feature])
        ev_df[time_feature] = ev_df[time_feature].apply(lambda x: x.tz_localize(None).tz_localize('UTC'))

            # Step d.b event: Generate dataframes with samples_from_df_ev function
        v_odfs, v_pdfs = samples_from_df_ev(
                ev_df, history[0], history[1], main_feature,
                time_feature, group_feature,
                value_feature, main_value, slicer_params
            )

        all_odfs.extend(v_odfs)
        all_pdfs[group_feature] = v_pdfs

    # Step e: Concatenate all the returned "odfs" and "odfs_ev" in a ordered DataFrame
    final_odf_df = pd.concat(all_odfs).sort_values(by=["i", "vt"]).reset_index(drop=True)

    return final_odf_df, all_pdfs


#### Applying samples_from_dataset on all the CSV files

In [236]:
main_value = "p01"
main_feature = "participant"
time_feature = "vt"

timeseries_df_list = {"paths": ["datasets/fitbit/processed_data/calories.csv",
                                "datasets/fitbit/processed_data/distance.csv",
                                "datasets/fitbit/processed_data/steps.csv",
                                "datasets/fitbit/processed_data/heart_rate.csv"],
                       "group_features": ["calories", "distance", "steps", "bpm"],
                       "bucket_values": [[1.16, 1.32, 1.59], [1236], [15.9], [51, 63, 76]]}

event_df_list = {"paths": ["datasets/fitbit/processed_data/exercise.csv",
                            "datasets/fitbit/processed_data/sleep_score.csv",
                            "datasets/fitbit/processed_data/sleep.csv"],
                  "group_features": ["Activity", "label", "sleep_phase"],
                  "value_feature": ["value", "value", "value"]}

slicer_params = {"stride": 24, "ow": 20, "ww": 4, "pw": 48, "granularity": "hours"}

final_odf, final_pdf = samples_from_dataset(main_value, main_feature, time_feature, timeseries_df_list, event_df_list, slicer_params)

100%|██████████| 149/149 [00:00<00:00, 1294.64it/s]


### Scatter Plot

In [237]:
#!pip install plotly
import plotly.express as px

def show_sequence(odf, i):

    data = odf[odf["i"] == i]
        
    fig = px.scatter(data, x="vt", y="label", size="value", color="label", labels={"label": "Label", "vt": "Timestamp in seconds", "value": "value"}, 
                        template="plotly_dark", width=800, height=400)
        
    fig.update_layout(showlegend=True, legend=dict(title_text="Label Legend", orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0))
    fig.update_traces(marker=dict(line=dict(width=1, color="white")), selector=dict(mode='markers'))

    fig.show()

show_sequence(final_odf, 3)


### Timeline

In [238]:
import plotly.express as px
def show_timeline_chart(odf, i):

    df = odf[odf["i"] == i].copy()

    # Calculate "end_time" and "duration" for rows where "vt" is not zero
    mask = df["vt"] != 0
    df.loc[mask, "end_time"] = df["vt"].shift(-1)
    df.loc[mask, "duration"] = df["end_time"] - df["vt"]
    # For rows where "vt" is zero, set "end_time" to the next non-zero "vt" and "duration" to 0
    df.loc[~mask, "end_time"] = df.loc[mask, "vt"].iloc[0]
    df.loc[~mask, "duration"] = df["end_time"] - df["vt"]

    df['vt'] = pd.to_datetime(df['vt'], unit='s')
    df['end_time'] = pd.to_datetime(df['end_time'], unit='s')
    df.fillna(method='ffill', inplace=True)

    data = df
    fig = px.timeline(data, x_start="vt", x_end="end_time", y="label", color="label", 
                      hover_data = "value", labels = "label", range_x = ["vt","end_time"])
    fig.update_layout(showlegend=True, legend=dict(title_text="Label Legend", orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0))
    fig.update_traces(marker=dict(line=dict(width=1, color="white")), selector=dict(mode='markers'))

    fig.show()

show_timeline_chart(final_odf, 85)


/var/folders/km/tq9jcfv91lqb_rwy4mw5cy780000gn/T/ipykernel_3173/1303515353.py:16: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



### Data Preprocesing

In [244]:
ddf = final_odf.copy() #[final_odf["i"] == 1].copy()

ddf['vt'] = pd.to_datetime(ddf['vt'], unit='s')
ddf['end_time'] = pd.to_datetime(ddf['vt'], unit='s')

# Calculate "end_time" and "duration" for rows where "vt" is not zero
mask = ddf["vt"] != 0
ddf.loc[mask, "end_time"] = ddf["vt"].shift(-1)
ddf.loc[mask, "duration"] = ddf["end_time"] - ddf["vt"]

# For rows where "vt" is zero, set "end_time" to the next non-zero "vt" and "duration" to 0
ddf.loc[~mask, "end_time"] = ddf.loc[mask, "vt"].iloc[0]
ddf.loc[~mask, "duration"] = ddf["end_time"] - ddf["vt"]

ddf.fillna(method='ffill', inplace=True)

#putting last i's "end_time" one minute after the "vt"
for unique_i in ddf['i'].unique():
    # Create a mask for the specific value of "i"
    last_row_mask = (ddf['i'] == unique_i) & (ddf.index == ddf[ddf['i'] == unique_i].index[-1])
    ddf.loc[last_row_mask, "end_time"] = ddf.loc[last_row_mask, "vt"]
    ddf.loc[last_row_mask, "end_time"] += pd.Timedelta(seconds=60)
    ddf.loc[last_row_mask, "duration"] = ddf["end_time"] - ddf["vt"]


ddf.to_csv("datasets/fitbit/processed_data/p01_odf.csv")

ddf['end_time'] = pd.to_datetime(ddf['end_time'], unit='s')

ddf.info(), print(ddf)


/var/folders/km/tq9jcfv91lqb_rwy4mw5cy780000gn/T/ipykernel_3173/1431922604.py:15: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7744 entries, 0 to 7743
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   i         7744 non-null   int64          
 1   vt        7744 non-null   datetime64[ns] 
 2   label     7744 non-null   object         
 3   value     7744 non-null   float64        
 4   end_time  7744 non-null   datetime64[ns] 
 5   duration  7744 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(1), timedelta64[ns](1)
memory usage: 363.1+ KB
        i                  vt       label    value            end_time  \
0       0 1970-01-01 00:00:00  calories_3  28140.0 1970-01-01 00:00:00   
1       0 1970-01-01 00:00:00  distance_1  24480.0 1970-01-01 00:00:00   
2       0 1970-01-01 00:00:00     steps_1  24480.0 1970-01-01 00:00:00   
3       0 1970-01-01 00:00:00       bpm_2  25260.0 1970-01-01 00:46:00   
4       0 1970-01-01 00:46:00       bpm_3  13

(None, None)

In [2]:
import pandas as pd
pd.read_csv("p01.csv")

,Unnamed: 0,sequence,class
0,0,"[(0.0, 'calories_3', 28140.0), (0.0, 'distance...",True
1,1,"[(0.0, 'calories_3', 26760.0), (0.0, 'distance...",False
2,2,"[(0.0, 'calories_4', 14160.0), (0.0, 'distance...",False
3,3,"[(0.0, 'calories_4', 9000.0), (0.0, 'distance_...",True
4,4,"[(0.0, 'calories_3', 23640.0), (0.0, 'distance...",False
...,...,...,...
144,144,"[(0.0, 'calories_3', 30660.0), (0.0, 'distance...",False
145,145,"[(0.0, 'calories_3', 31380.0), (0.0, 'distance...",True
146,146,"[(0.0, 'calories_4', 540.0), (0.0, 'distance_1...",True
147,147,"[(0.0, 'calories_3', 35280.0), (0.0, 'distance...",False
